<a href="https://colab.research.google.com/github/SohilK03/Machine_Learning_Projects/blob/main/Facial_Emotion_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/muxspace/facial_expressions.git

Cloning into 'facial_expressions'...
remote: Enumerating objects: 14214, done.
remote: Total 14214 (delta 0), reused 0 (delta 0), pack-reused 14214
Receiving objects: 100% (14214/14214), 239.65 MiB | 43.70 MiB/s, done.
Resolving deltas: 100% (223/223), done.
Checking out files: 100% (13996/13996), done.


In [11]:
import csv
di={}
with open ("/content/facial_expressions/data/legend.csv") as f:
  reader=csv.reader(f)
  next(reader)
  for row in reader:
    if row[2].lower() in di:
      di[row[2].lower()].append(row[1])
    else:
      di[row[2].lower()]=[row[1]]


In [12]:
emotion_list=list(di.keys())
emotion_list

['anger',
 'surprise',
 'disgust',
 'fear',
 'neutral',
 'happiness',
 'sadness',
 'contempt']

In [13]:
import os
os.mkdir('master_data')
os.mkdir("master_data/training")
os.mkdir("master_data/testing")

In [15]:
for emotion in emotion_list:
  os.mkdir(os.path.join('master_data/training/',emotion))
  os.mkdir(os.path.join('master_data/testing/',emotion))

In [22]:
from shutil import copyfile
split_size=0.8
for emotion,images in di.items():
  train_size=int(split_size*len(images))
  train_data=images[:train_size]
  test_data=images[train_size:]
  for image in train_data:
    source=os.path.join("/content/facial_expressions/images/",image)
    dest=os.path.join("/content/master_data/training/",emotion,image)
    copyfile(source,dest)
  for image in test_data:
    source=os.path.join("/content/facial_expressions/images/",image)
    dest=os.path.join("/content/master_data/testing/",emotion,image)
    copyfile(source,dest)


In [24]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dense,Flatten

In [35]:
model=tf.keras.models.Sequential(
    [Conv2D(16,(3,3),activation="relu",input_shape=(100,100,3)),
     MaxPooling2D(2,2),
     Conv2D(32,(3,3),activation="relu"),
     MaxPooling2D(2,2),
     Conv2D(64,(3,3),activation="relu"),
     MaxPooling2D(2,2),
     Flatten(),
     Dense(1024,activation="relu"),
     Dense(8,activation="softmax")]
     )

In [36]:
model.compile(optimizer=Adam(lr=0.01),loss="categorical_crossentropy",metrics=['acc'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 98, 98, 16)        448       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 49, 49, 16)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 47, 47, 32)        4640      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 21, 21, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6400)             

In [29]:
train_dir="/content/master_data/training"
test_dir="/content/master_data/testing"

train_datagen=ImageDataGenerator(rescale=1.0/255)
train_generator=train_datagen.flow_from_directory(
    train_dir,target_size=(100,100),class_mode='categorical',batch_size=128
)
test_datagen=ImageDataGenerator(rescale=1.0/255)
test_generator=test_datagen.flow_from_directory(
    test_dir,target_size=(100,100),class_mode='categorical',batch_size=128
)


Found 10941 images belonging to 8 classes.
Found 2742 images belonging to 8 classes.


In [30]:
es=EarlyStopping(monitor='val_acc',patience=2,min_delta=0.01)

In [37]:
model.fit_generator(train_generator,epochs=10,verbose=1,validation_data=test_generator,callbacks=[es])

Epoch 1/10
86/86 [==============================] - 15s 174ms/step - loss: 1.6081 - acc: 0.4825 - val_loss: 1.0428 - val_acc: 0.5193
Epoch 2/10
86/86 [==============================] - 15s 170ms/step - loss: 1.0072 - acc: 0.5439 - val_loss: 0.9905 - val_acc: 0.5941
Epoch 3/10
86/86 [==============================] - 15s 171ms/step - loss: 0.8693 - acc: 0.6823 - val_loss: 0.9173 - val_acc: 0.6641
Epoch 4/10
86/86 [==============================] - 15s 171ms/step - loss: 0.7602 - acc: 0.7324 - val_loss: 0.9004 - val_acc: 0.6623
Epoch 5/10
86/86 [==============================] - 15s 173ms/step - loss: 0.7080 - acc: 0.7506 - val_loss: 1.1517 - val_acc: 0.5846
